In [2]:
import pandas as pd

In [16]:
df = pd.read_csv(r"IndovestDKG_NORMALIZED_ENTITY_HIGH-O.csv")
df

,subject,subject_type,relation,object,object_type,date
0,manulife aset manajemen,PERUSAHAAN,Mengumumkan,diversifikasi investasi,KONSEP,2025-03-07 21:18:00
1,ihsg,INDIKATOR_EKONOMI,Menghasilkan,pertumbuhan tahunan tertinggi,INDIKATOR_EKONOMI,2025-03-07 21:18:00
2,strategi diversifikasi,KONSEP,Meningkatkan,peluang return,KONSEP,2025-03-07 21:18:00
3,investor,ORANG,Mengendalikan,berbagai jenis aset investasi,KONSEP,2025-03-07 21:18:00
4,dimas ardinugraha,ORANG,Mempengaruhi,strategi diversifikasi,KONSEP,2025-03-07 21:18:00
...,...,...,...,...,...,...
68202,mahkamah konstitusi,BADAN_REGULATOR,Mengadakan,sidang,EVENT,2017-09-26 19:37:00
68203,investasi keuangan,KONSEP,Mempengaruhi,jemaah haji,ORANG,2017-09-26 19:37:00
68204,kementerian agama,PEMERINTAHAN,Mengendalikan,pengelolaan keuangan haji,KONSEP,2017-09-26 19:37:00
68205,kementerian agama,PEMERINTAHAN,Menghasilkan,perbankan syariah,SEKTOR,2017-09-26 19:37:00


In [11]:
import pandas as pd
import numpy as np

STATUS_DATASET = "norm_IndovestDKG"

df['date'] = pd.to_datetime(df['date'])
df.sort_values("date", ascending=False, inplace=True)

max_date = df['date'].max()
df['temporal_id'] = ((max_date - df['date']).dt.days // 7).astype(int)

subjects = df[['subject', 'subject_type']].rename(columns={'subject': 'entity_name', 'subject_type': 'entity_type'})
objects  = df[['object', 'object_type']].rename(columns={'object': 'entity_name', 'object_type': 'entity_type'})
entities = pd.concat([subjects, objects]).drop_duplicates(subset='entity_name')

entities = entities.reset_index(drop=True)
entities['entity_id'] = entities.index

unique_types = entities['entity_type'].unique()
type2id = {etype: idx for idx, etype in enumerate(unique_types)}
entities['entity_type_id'] = entities['entity_type'].map(type2id)

encoded_entity = entities[['entity_name', 'entity_id', 'entity_type', 'entity_type_id']]
encoded_entity.columns = ['entity_name', 'entity_id', 'relation', 'relation_id']

unique_relations = df['relation'].unique()
relation2id = {rel: idx for idx, rel in enumerate(unique_relations)}
encoded_relation = pd.DataFrame(list(relation2id.items()), columns=['relation', 'relation_id'])

entity2id = entities.set_index('entity_name')['entity_id'].to_dict()

df['subject_entity_id'] = df['subject'].map(entity2id)
df['object_entity_id']  = df['object'].map(entity2id)
df['relation_id'] = df['relation'].map(relation2id)

encoded_indovest = df[['subject_entity_id', 'relation_id', 'object_entity_id', 'temporal_id']]

encoded, entity_id, relation_id = f"result/{STATUS_DATASET}_encoded_indovest.csv", f"result/{STATUS_DATASET}_encoded_entity_id.csv", f"result/{STATUS_DATASET}_relation_id.csv"

encoded_indovest.to_csv(encoded, index=False)
encoded_entity.to_csv(entity_id, index=False)
encoded_relation.to_csv(relation_id, index=False)

print("Encoding complete. Files saved: encoded_indovest.csv, encoded_entity_id.csv, encoded_relation_id.csv")


Encoding complete. Files saved: encoded_indovest.csv, encoded_entity_id.csv, encoded_relation_id.csv


encode timestamp per 1 minggu

In [13]:
encoded_indovest

,subject_entity_id,relation_id,object_entity_id,temporal_id
0,0,0,10203,0
6,1,1,244,0
10,1,1,16606,0
9,1,1,16607,0
8,1,1,3573,0
...,...,...,...,...
68202,10426,53,32353,388
68203,5343,4,35284,388
68204,2002,2,35285,388
68205,2002,1,15919,388


In [14]:
encoded_entity

,entity_name,entity_id,relation,relation_id
0,manulife aset manajemen,0,PERUSAHAAN,0
1,dimas ardinugraha,1,ORANG,1
2,ihsg,2,INDIKATOR_EKONOMI,2
3,indeks bindo,3,INDIKATOR_EKONOMI,2
4,investor,4,ORANG,1
...,...,...,...,...
45304,200 unit apartemen,45304,PRODUK,6
45305,orang jakarta,45305,ORANG,1
45306,apartemen harga terjangkau,45306,PRODUK,6
45307,"""kawasan perkotaan""",45307,KONSEP,3


In [15]:
encoded_relation

,relation,relation_id
0,Mengumumkan,0
1,Menghasilkan,1
2,Mengendalikan,2
3,Meningkatkan,3
4,Mempengaruhi,4
...,...,...
1184,Direkomendasikan Untuk,1184
1185,Akan membayar,1185
1186,Akan diterapkan,1186
1187,Dapat mencoba,1187
